DATA643- Project 2
======
Prashanth Padebetttu, Parshu Rath, Adjare Windokun, Xingjia Wu
----------
<b> Summer 2016 <br>
Instructor: Andrew Catlin

Load Movielens Data
-------
<b> Load 100K movie data as pandas dataframe:

In [221]:
# Required Python Packages
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity
import csv
import pandas as pd
import numpy as np
from scipy import spatial

In [222]:
#Load data into a pandas dataframe
# Movielens data with 100,000 records
url = 'https://raw.githubusercontent.com/ppadebettu/CUNY/IS-607-Data-Acquisition-and-Management/IS643/Project_2/movielens.csv'
df = pd.read_csv(url, sep = "," , header = 0, na_values='NaN')


In [223]:
df.head()

,userid,age,gender,occupation,zipcode,itemid,rating,timestamp,movieid,movietitle,...,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western
0,196,49,M,writer,55105,242,3,881250949,242,Kolya (1996),...,0,0,0,0,0,0,0,0,0,0
1,186,39,F,executive,00000,302,3,891717742,302,L.A. Confidential (1997),...,0,1,0,0,1,0,0,1,0,0
2,22,25,M,writer,40206,377,1,878887116,377,Heavyweights (1994),...,0,0,0,0,0,0,0,0,0,0
3,244,28,M,technician,80525,51,2,880606923,51,Legends of the Fall (1994),...,0,0,0,0,0,1,0,0,1,1
4,166,47,M,educator,55113,346,1,886397596,346,Jackie Brown (1997),...,0,0,0,0,0,0,0,0,0,0


<b>Some data exploration:

In [224]:
#What are the 25 most rated movies?
most_rated = df.groupby('movietitle').size().sort_values(ascending=False)[:10]
most_rated

movietitle
Star Wars (1977)                 583
Contact (1997)                   509
Fargo (1996)                     508
Return of the Jedi (1983)        507
Liar Liar (1997)                 485
English Patient, The (1996)      481
Scream (1996)                    478
Toy Story (1995)                 452
Air Force One (1997)             431
Independence Day (ID4) (1996)    429
dtype: int64

In [225]:
#Which movies are highly rated?
movie_stats = df.groupby('movietitle').agg({'rating': [np.size, np.mean]})
movie_stats.head()

rating          
                            size      mean
movietitle                                
'Til There Was You (1997)      9  2.333333
1-900 (1994)                   5  2.600000
101 Dalmatians (1996)        109  2.908257
12 Angry Men (1957)          125  4.344000
187 (1997)                    41  3.024390

In [226]:
#Sort by movie rating average
movie_stats.sort_values([('rating', 'mean')], ascending=False).head()

rating     
                                                    size mean
movietitle                                                   
Aiqing wansui (1994)                                   1  5.0
Prefontaine (1997)                                     3  5.0
Entertaining Angels: The Dorothy Day Story (1996)      1  5.0
Star Kid (1997)                                        3  5.0
Saint of Fort Washington, The (1993)                   2  5.0

In [227]:
#Movies that have been rated at least 100 times.
atleast_100 = movie_stats['rating']['size'] >= 100
movie_stats[atleast_100].sort_values([('rating', 'mean')], ascending=False)[:5]

rating          
                                   size      mean
movietitle                                       
Close Shave, A (1995)               112  4.491071
Schindler's List (1993)             298  4.466443
Wrong Trousers, The (1993)          118  4.466102
Casablanca (1942)                   243  4.456790
Shawshank Redemption, The (1994)    283  4.445230

<b> Load 100K movie data as Python dictionary:

In [228]:
# Get movie titles
import urllib2
url = 'https://raw.githubusercontent.com/prcuny/DATA643/master/Project-2/'
movies={}
response = urllib2.urlopen(url + 'u.item')
for line in response:    
        (id,title)=line.split('|')[0:2]
        movies[id]=title


In [229]:
# Load user ratings
mdict={}
response = urllib2.urlopen(url + 'u.data')
for line in response:
       (user,movieid,rating,ts)=line.split('\t')
       mdict.setdefault(user,{})
       mdict[user][movies[movieid]]=float(rating)



<b>Some data exploration:
</b><br>
Print first 2 users and ratings

In [230]:
#Adapted from http://stackoverflow.com/questions/7971618/python-return-first-n-keyvalue-pairs-from-dict
from itertools import islice
def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n ))
#take(2, mdict.iteritems())  
#Too Long output. 
#Try to output one user's ratings instead
#user '20' and ratings
#mdict['20']



<b> Generate dataframe from dictionary

In [231]:
df_fromDict = pd.DataFrame.from_dict(mdict, orient='index')
df_fromDict.head()

,"Birdcage, The (1996)",Enchanted April (1991),Diabolique (1996),Face/Off (1997),My Fellow Americans (1996),In the Line of Fire (1993),Murder at 1600 (1997),Star Trek: The Wrath of Khan (1982),Grosse Pointe Blank (1997),Courage Under Fire (1996),...,Raw Deal (1948),Shopping (1994),Nemesis 2: Nebula (1995),Wings of Courage (1995),Mad Dog Time (1996),Mamma Roma (1962),"New Age, The (1994)",Target (1995),Johns (1996),Tokyo Fist (1995)
1,4.0,NaN,4.0,NaN,NaN,NaN,NaN,5.0,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
102,NaN,NaN,NaN,3.0,NaN,2.0,3.0,4.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Normalization with center mean

- Normalized dataframe and fill NaN with 0

In [232]:
df_norm1 = df_fromDict.apply(lambda x: x - x.mean(), axis =1) # row-wise operation

In [233]:
df_norm = df_norm1.fillna(0) #fill NaN with 0
df_norm.head()

,"Birdcage, The (1996)",Enchanted April (1991),Diabolique (1996),Face/Off (1997),My Fellow Americans (1996),In the Line of Fire (1993),Murder at 1600 (1997),Star Trek: The Wrath of Khan (1982),Grosse Pointe Blank (1997),Courage Under Fire (1996),...,Raw Deal (1948),Shopping (1994),Nemesis 2: Nebula (1995),Wings of Courage (1995),Mad Dog Time (1996),Mamma Roma (1962),"New Age, The (1994)",Target (1995),Johns (1996),Tokyo Fist (1995)
1,0.394834,0.000000,0.394834,0.000000,0.0,0.000000,0.000000,1.394834,0.394834,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,0.000000,0.793478,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.104478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
102,0.000000,0.000000,0.000000,0.384259,0.0,-0.615741,0.384259,1.384259,0.384259,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


- Normalized dictionary with NaN removed

In [234]:
dict_norm1 = df_norm1.to_dict(orient = "records") # list of dictionary

In [235]:
dict_norm2 = {df_norm.index[i]: dict_norm1[i] for i in range(len(dict_norm1))} # dict of dict

In [236]:
# Remove NaN 
from math import isnan
dict_norm = {}
# count = 0
for key, value in dict_norm2.iteritems():
    dict_temp = {}
    for key1, value1 in dict_norm2[key].iteritems():
        if not isnan(value1):
            dict_temp[key1] = value1
#     if count==0:
#         print len(dict_temp)
#         print key
#     count = 1
    dict_norm[key] = dict_temp

### User-User Collaborative Filtering
### UUCF using python dictionry

<b> Cosine Distance (User to User)

In [237]:
#Function that returns a cosine-based similarity score for person1 and person2
def sim_cosine(dict, p1,p2):
#Get the list of mutually rated items, ignore items that were not rated by either person
 si = {}
 p1set = []
 p2set = []
 for item in dict[p1]:
       if (item in dict[p2]) and not(np.isnan(dict[p2][item])) and not(np.isnan(dict[p1][item])) :
          si[item] = 1
#If they have no rating in common, return 0
 if len(si) == 0: 
  return 0
 p1set = [dict[p1][item] for item in si.keys()]
 p2set = [dict[p2][item] for item in si.keys()]
 return round((1 - spatial.distance.cosine(p1set, p2set)), 4)


In [238]:
# Test the Pearson-based similarity function
round(sim_cosine(mdict,'20','30'), 4) # With orginal data

0.8991

In [239]:
round(sim_cosine(dict_norm,'20','30'), 4) # With normalized data

-0.4126

<b> UUCF (python dictionary) Recommendations using cosine distance weighting: 

In [240]:
#Function that returns 'n' recommendations for a person by using a weighted average of every other user's rankings

def getRecommendations(n, dict,person,similarity=sim_cosine):
 totals = {}
 simSums = {}

 for other in dict:
  #don't compare me to myself
  if other == person:
   continue
  sim = similarity(dict,person,other)
  #ignore scores of zero or lower
  if sim <= 0: 
   continue
  for item in dict[other]:
   #only score movies I haven't rated yet
   if (item not in dict[person]) or (dict[person][item] == 0) or (np.isnan(dict[person][item])) :
    #Similarity * score
    totals.setdefault(item,0)
    totals[item] += not(np.isnan(dict[other][item] * sim))
    #Sum of similarities
    simSums.setdefault(item,0)
    simSums[item] += sim

    #Create the normalized list
 rankings = [(total/simSums[item],item) for item,total in totals.items()]

 #Return the sorted list
 rankings.sort()
 rankings.reverse()
 return rankings[0:n]


<b> Get top 10 recommendations for user '20' using User-User CF

In [241]:
#Test getRecommendations function
getRecommendations(10, mdict,'20',similarity=sim_cosine) # with original data

[(1.4894250819183794, 'Mighty, The (1998)'),
 (1.4215651432226881, 'Star Kid (1997)'),
 (1.3932427725531173, 'Men of Means (1998)'),
 (1.3415015874435452, 'Ill Gotten Gains (1997)'),
 (1.3147515119642388, 'Boys, Les (1997)'),
 (1.301720130172013, 'Hugo Pool (1997)'),
 (1.3003055718093752, 'Leading Man, The (1996)'),
 (1.2743723716069837, 'Shooting Fish (1997)'),
 (1.2723725506828398, 'The Innocent (1994)'),
 (1.266079205915122, '3 Ninjas: High Noon At Mega Mountain (1998)')]

In [242]:
getRecommendations(10, dict_norm,'20',similarity=sim_cosine) # with normalized data

[(588.2352941176471, 'Favor, The (1994)'),
 (123.4567901234568, 'Saint of Fort Washington, The (1993)'),
 (93.45794392523365, 'My Favorite Season (1993)'),
 (93.45794392523365, 'Magic Hour, The (1998)'),
 (93.45794392523365, 'Little City (1998)'),
 (76.33587786259541, 'Quiet Room, The (1996)'),
 (53.191489361702125, 'Reckless (1995)'),
 (52.631578947368425, 'Female Perversions (1996)'),
 (52.631578947368425, 'Commandments (1997)'),
 (52.631578947368425, 'Bad Girls (1994)')]

### UU-CF using pandas dataframe

In [243]:
# function takes in a data frame, what you want to predict pred_id, 
#the type of collaboration, the similarity you want
# to use, and the number of responses you want to get back
## Code adapted from https://blog.liip.ch/archive/2016/06/16/recommender-system-slack-pandas-flask.html

def predict_p(dframe, pred_id, kind = 'user', similarity = cosine_similarity, n = 10):
    d = dframe.fillna(0)
    if kind == 'user':
        dists = similarity(d)
        dists_named = pd.DataFrame(dists, columns = d.index)
        dists_named.index = dists_named.columns
        
    elif kind == 'item':
        d = d.T
        dists = similarity(d)
        dists_named = pd.DataFrame(dists, columns = d.index)
        dists_named.index = dists_named.columns

    p = dists_named[pred_id].sort_values(ascending = False)
    mask = p.index.isin([pred_id])
    p = p[~mask]
    return p[:n]   

In [244]:
df.head()

,userid,age,gender,occupation,zipcode,itemid,rating,timestamp,movieid,movietitle,...,fantasy,filmnoir,horror,musical,mystery,romance,scifi,thriller,war,western
0,196,49,M,writer,55105,242,3,881250949,242,Kolya (1996),...,0,0,0,0,0,0,0,0,0,0
1,186,39,F,executive,00000,302,3,891717742,302,L.A. Confidential (1997),...,0,1,0,0,1,0,0,1,0,0
2,22,25,M,writer,40206,377,1,878887116,377,Heavyweights (1994),...,0,0,0,0,0,0,0,0,0,0
3,244,28,M,technician,80525,51,2,880606923,51,Legends of the Fall (1994),...,0,0,0,0,0,1,0,0,1,1
4,166,47,M,educator,55113,346,1,886397596,346,Jackie Brown (1997),...,0,0,0,0,0,0,0,0,0,0


In [245]:
# convert to wide format for analysis
d = df.pivot_table(index = 'userid', columns = 'movietitle', values = 'rating')
del d.index.name

# normalized dataframe
d_norm1 = d.apply(lambda x: x - x.mean(), axis =1) # row-wise operation
d_norm = d_norm1.fillna(0)

# original dataframe
#d = d.fillna(0)
d.head()

movietitle,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown
1,NaN,NaN,2.0,5.0,NaN,NaN,3.0,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,5.0,3.0,NaN,NaN,NaN,4.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,4.0


In [246]:
d_norm.head()

movietitle,'Til There Was You (1997),1-900 (1994),101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",...,Wyatt Earp (1994),Yankee Zulu (1994),Year of the Horse (1997),You So Crazy (1994),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown
1,0.0,0.0,-1.605166,1.394834,0.000000,0.0,-0.605166,0.394834,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.394834,-0.605166,0.0,0.0,0.0,0.394834
2,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-2.704918,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.000000,0.000000,-0.773585,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000
5,0.0,0.0,-0.874286,0.000000,0.000000,0.0,0.000000,1.125714,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.125714,0.000000,0.0,0.0,0.0,1.125714


<b> User-User Collaborative Filtering using cosine distance

<b>Predictions based on user-user cosine distance

In [247]:
# Predict based on user/user
## At present this gives the closest users
predict_p(d, pred_id = 290, kind = 'user') # With original data

254    0.575197
738    0.568915
881    0.565369
42     0.547329
545    0.543806
804    0.543751
650    0.543560
275    0.540358
815    0.536989
536    0.532302
Name: 290, dtype: float64

In [248]:
predict_p(d_norm, pred_id = 290, kind = 'user') # With normalized data

70     0.323139
275    0.279290
746    0.233377
536    0.233276
901    0.231759
638    0.231161
545    0.219109
896    0.218910
807    0.216560
56     0.215462
Name: 290, dtype: float64

<b> UU-CF (pandas) Recommendations

In [249]:
# Recommend items based on User similarity
def get_recommendations_user(dframe, pred_id, similarity = cosine_similarity, n = 10):
    d = dframe.fillna(0)
    t = d.ix[predict_p(d, pred_id = pred_id, n = n).index]
    #t.ix[:,:10]
    p = d.ix[pred_id].values != 0



    r = t.T.ix[~p].T
    # Delete those columns that all contain all 0, meaning that they have not been rated
    r = r.loc[:, (r != 0).any(axis=0)]
    # Have to rename the columns as we keep on getting an error
    cols = r.columns
    r.columns =  range(1, 1+ r.shape[1])
    # Calculate the mean ratings for the movies
    ratings = r.sum(axis = 0)/r.astype(bool).sum(axis= 0)
    ratings.index = cols
    return ratings.sort_values(axis = 0, ascending = False, 
                inplace = False, na_position = 'last')[:n]
    

<b> Get top 10 recommendations (UUCF/pandas)

In [250]:
get_recommendations_user(d, pred_id = 290, similarity = cosine_similarity, n = 10) # With original data

movietitle
Enchanted April (1991)          5.0
Ran (1985)                      5.0
Perfect World, A (1993)         5.0
Persuasion (1995)               5.0
Little Princess, A (1995)       5.0
My Man Godfrey (1936)           5.0
Shadowlands (1993)              5.0
Sense and Sensibility (1995)    5.0
Army of Darkness (1993)         5.0
That Thing You Do! (1996)       5.0
dtype: float64

In [251]:
# Check the results
cut = [254, 738,881, 42,545,804,650,275,815,536]
d.loc[cut,'Enchanted April (1991)']


254    NaN
738    NaN
881    NaN
42     NaN
545    NaN
804    NaN
650    NaN
275    NaN
815    NaN
536    5.0
Name: Enchanted April (1991), dtype: float64

In [252]:
get_recommendations_user(d_norm, pred_id = 290, similarity = cosine_similarity, n = 10) # With normalized data

movietitle
Cop Land (1997)                   2.019337
Citizen Kane (1941)               2.019337
Fried Green Tomatoes (1991)       2.019337
Philadelphia Story, The (1940)    2.019337
Devil in a Blue Dress (1995)      2.019337
Heat (1995)                       2.019337
Big Sleep, The (1946)             2.019337
Henry V (1989)                    1.572940
Unforgiven (1992)                 1.251905
Twelve Monkeys (1995)             1.188813
dtype: float64

In [253]:
r = pd.DataFrame(get_recommendations_user(d_norm, pred_id = 290, similarity = cosine_similarity, n = 10))
# With normalized data
mean_ratings = d[d.index ==209].mean(axis = 1)

r + float(mean_ratings)



,0
movietitle,
Cop Land (1997),5.201155
Citizen Kane (1941),5.201155
Fried Green Tomatoes (1991),5.201155
"Philadelphia Story, The (1940)",5.201155
Devil in a Blue Dress (1995),5.201155
Heat (1995),5.201155
"Big Sleep, The (1946)",5.201155
Henry V (1989),4.754758
Unforgiven (1992),4.433723


### Item-Item Collaborative Filtering
### II-CF using python dictionry

<b> Transpose the user-item matrix to item-user matrix

In [254]:
#Flip items and users
def item_user_switch(df):
    results={}
    for uid in df:
        for movie in df[uid]:
            results.setdefault(movie, {})
            #Flip movie & user
            results[movie][uid]=df[uid][movie]
    return results

In [255]:
#Tranpose the matrix
mdict_rev = item_user_switch(mdict)

In [256]:
#Check the data
#mdict_rev['Forrest Gump (1994)']

In [257]:
#Test getRecommendations function
#getRecommendations(10, mdict_rev,'20',similarity=sim_cosine)  #Not working

### II-CF using pandas dataframe Un Normalized Data

In [258]:
# Predict based on item/item
predict_p(d, pred_id = "Amadeus (1984)", kind = 'item')

movietitle
Forrest Gump (1994)                       0.661877
Raiders of the Lost Ark (1981)            0.660880
Back to the Future (1985)                 0.643994
E.T. the Extra-Terrestrial (1982)         0.637103
Dead Poets Society (1989)                 0.636788
One Flew Over the Cuckoo's Nest (1975)    0.636475
Silence of the Lambs, The (1991)          0.628871
2001: A Space Odyssey (1968)              0.627704
When Harry Met Sally... (1989)            0.627460
Schindler's List (1993)                   0.617628
Name: Amadeus (1984), dtype: float64

### II-CF (pandas) Recommendations Normalized Data

In [259]:
predict_p(d_norm, pred_id = "Amadeus (1984)", kind = 'item')

movietitle
Schindler's List (1993)                   0.398994
Casablanca (1942)                         0.344384
To Kill a Mockingbird (1962)              0.331411
Citizen Kane (1941)                       0.328816
Shawshank Redemption, The (1994)          0.326825
One Flew Over the Cuckoo's Nest (1975)    0.320654
Godfather, The (1972)                     0.309454
Henry V (1989)                            0.296642
Silence of the Lambs, The (1991)          0.287307
Star Wars (1977)                          0.280790
Name: Amadeus (1984), dtype: float64

### Recommender Evaluation

<b>Split the dataset into training and testing sets

In [260]:
from sklearn import cross_validation as cv
df1 = df[["userid", "itemid", "rating"]]
df1.columns = ["userid", "itemid", "rating"]
train_data, test_data = cv.train_test_split(df1, test_size=0.25)

In [261]:
# Normalized long format
d1 = df.pivot_table(index = 'userid', columns = 'movieid', values = 'rating')
del d1.index.name
d_norm2 = d1.apply(lambda x: x - x.mean(), axis =1) 
df_long_norm = pd.melt(d_norm2.reset_index(), id_vars = "index").dropna()
df_long_norm.columns = ["userid", "itemid", "rating"]

In [262]:
df_long_norm.head()

,userid,itemid,rating
0,1,1,1.389706
1,2,1,0.290323
4,5,1,1.125714
5,6,1,0.364929
9,10,1,-0.206522


In [263]:
n_users = df1.userid.unique().shape[0]
n_items = df1.itemid.unique().shape[0]
print 'Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_items)  

Number of users = 943 | Number of movies = 1682


In [264]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

In [265]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')


In [266]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis]) 
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])     
    return pred

In [267]:
train_data_matrix

array([[ 0.,  3.,  4., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [268]:
item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [269]:
user_prediction

array([[ 1.5755265 ,  0.52051391,  0.41531044, ...,  0.24206101,
         0.2419915 ,  0.23957549],
       [ 1.32083482,  0.26927465,  0.12335188, ..., -0.07730101,
        -0.07586136, -0.07907648],
       [ 1.35242894,  0.24002565,  0.10468518, ..., -0.10210179,
        -0.10043139, -0.10359531],
       ..., 
       [ 1.25864857,  0.21124912,  0.07107323, ..., -0.12128768,
        -0.1203953 , -0.12354972],
       [ 1.39168883,  0.31132466,  0.19693475, ..., -0.00381547,
        -0.00311829, -0.00588332],
       [ 1.4080473 ,  0.36692713,  0.26138491, ...,  0.0847689 ,
         0.08455932,  0.08228216]])

<b>Evaluation

In [270]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, actual):
    prediction = prediction[actual.nonzero()].flatten() 
    actual = actual[actual.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, actual))

In [271]:
print 'User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix))
print 'Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix))

User-based CF RMSE: 3.13291685691
Item-based CF RMSE: 3.46114362212
